In [1]:
!git clone https://github.com/your-username/claim_fraud_detection.git
%cd claim_fraud_detection

fatal: destination path 'claim_fraud_detection' already exists and is not an empty directory.
/content/claim_fraud_detection


In [16]:
!git pull origin main

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 2), reused 4 (delta 2), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 362 bytes | 10.00 KiB/s, done.
From https://github.com/amannain122/claim_fraud_detection
 * branch            main       -> FETCH_HEAD
   538c2e5..e2fc9e1  main       -> origin/main
Updating 538c2e5..e2fc9e1
Fast-forward
 data/dataset_raw | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


In [13]:
from google.colab import auth
auth.authenticate_user()

In [14]:
from google.cloud import storage

# Create the Google Cloud Storage client
client = storage.Client()

# Specify your bucket name
bucket_name = "claims_fraud_data"
bucket = client.get_bucket(bucket_name)

In [19]:
from scipy.io import arff
import pandas as pd

# Load ARFF file
data_path = '/content/claim_fraud_detection/data/dataset_raw'

# Attempt to load the ARFF file directly
try:
    data, meta = arff.loadarff(data_path)
    # Convert the loaded data into a DataFrame
    df = pd.DataFrame(data)

except Exception as e:
    print(f"Error loading ARFF file: {e}")

df.head()

,Upper_Age,Lower_Age,Reco_Policy_Premium,City_Code,Accomodation_Type,Reco_Insurance_Type,Is_Spouse,Health_Indicator,Holding_Policy_Duration,Holding_Policy_Type,class
0,52.0,52.0,16200.0,b'C2',b'Owned',b'Individual',b'No',b'X4',b'6.0',b'4.0',0.0
1,67.0,67.0,16900.0,b'C17',b'Rented',b'Individual',b'No',b'X1',b'7.0',b'3.0',1.0
2,75.0,75.0,25668.0,b'C10',b'Owned',b'Individual',b'No',b'X3',b'3.0',b'1.0',0.0
3,60.0,57.0,17586.8,b'C26',b'Owned',b'Joint',b'Yes',b'X1',b'14+',b'1.0',0.0
4,35.0,35.0,12762.0,b'C12',b'Rented',b'Individual',b'No',b'X1',b'3.0',b'2.0',0.0


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23548 entries, 0 to 23547
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Upper_Age                23548 non-null  float64
 1   Lower_Age                23548 non-null  float64
 2   Reco_Policy_Premium      23548 non-null  float64
 3   City_Code                23548 non-null  object 
 4   Accomodation_Type        23548 non-null  object 
 5   Reco_Insurance_Type      23548 non-null  object 
 6   Is_Spouse                23548 non-null  object 
 7   Health_Indicator         23548 non-null  object 
 8   Holding_Policy_Duration  23548 non-null  object 
 9   Holding_Policy_Type      23548 non-null  object 
 10  class                    23548 non-null  float64
dtypes: float64(4), object(7)
memory usage: 2.0+ MB


In [23]:
# Define categorical columns that may contain byte literals
categorical_columns = ['City_Code', 'Accomodation_Type', 'Reco_Insurance_Type', 'Is_Spouse', 'Health_Indicator', 'Holding_Policy_Duration', 'Holding_Policy_Type']

df[categorical_columns] = df[categorical_columns].applymap(lambda x: x.decode() if isinstance(x, bytes) else x)

<ipython-input-23-c781f6a6f497>:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[categorical_columns] = df[categorical_columns].applymap(lambda x: x.decode() if isinstance(x, bytes) else x)


In [24]:
df.head()

,Upper_Age,Lower_Age,Reco_Policy_Premium,City_Code,Accomodation_Type,Reco_Insurance_Type,Is_Spouse,Health_Indicator,Holding_Policy_Duration,Holding_Policy_Type,class
0,52.0,52.0,16200.0,C2,Owned,Individual,No,X4,6.0,4.0,0.0
1,67.0,67.0,16900.0,C17,Rented,Individual,No,X1,7.0,3.0,1.0
2,75.0,75.0,25668.0,C10,Owned,Individual,No,X3,3.0,1.0,0.0
3,60.0,57.0,17586.8,C26,Owned,Joint,Yes,X1,14+,1.0,0.0
4,35.0,35.0,12762.0,C12,Rented,Individual,No,X1,3.0,2.0,0.0


In [25]:
df.isnull().sum()

,0
Upper_Age,0
Lower_Age,0
Reco_Policy_Premium,0
City_Code,0
Accomodation_Type,0
Reco_Insurance_Type,0
Is_Spouse,0
Health_Indicator,0
Holding_Policy_Duration,0
Holding_Policy_Type,0


,Upper_Age,Lower_Age,Reco_Policy_Premium,class
count,23548.000000,23548.000000,23548.000000,23548.000000
mean,48.864192,46.365381,15409.000161,0.242059
std,16.021466,16.578403,6416.327319,0.428339
min,21.000000,16.000000,3216.000000,0.000000
25%,35.000000,32.000000,10704.000000,0.000000
50%,49.000000,46.000000,14580.000000,0.000000
75%,62.000000,60.000000,19140.000000,0.000000
max,75.000000,75.000000,43350.400000,1.000000


In [30]:
# Save to CSV
df.to_csv('cleaned_claim_data.csv', index=False)

# Upload to GCS
blob = bucket.blob('cleaned_claim_data.csv')
blob.upload_from_filename('cleaned_claim_data.csv')